In [2]:
import pandas as pd
import random
import numpy as np
import pickle
import os
from lxml import html
import requests
from time import sleep
from functools import reduce
import warnings
warnings.filterwarnings("ignore")

filepath = os.path.abspath('')

In [3]:
#get historical data and figure out teamID dilemma
old_data = pd.read_csv('MNCAATourneyDetailedResults.csv')
idteams = pd.read_csv('MTeams.csv')
old_data = old_data.merge(idteams, 
                          left_on = 'WTeamID', 
                          right_on = 'TeamID')
old_data = old_data.merge(idteams, 
                          left_on = 'LTeamID', 
                          right_on = 'TeamID')

old_data = old_data[['Season',
          'WTeamID',
          'WScore',
          'LTeamID',
          'LScore',
          'TeamName_x',
         'TeamName_y']].sort_values(by = 'Season', ascending = True)
old_data = old_data.rename(columns={"TeamName_x": "Winning Team",
                   "TeamName_y": "Losing Team"})

#randomly put the teams in the Team 1 or Team 2 column 
#this is because we need to train the model on both wins and losses
old_data['Team 1'] = np.random.randint(2, size=len(old_data))
old_data['Team 2'] = 1 - old_data['Team 1']

#Now we can do some reformatting here...
conds_1 = [(old_data['Team 1'] == 1),(old_data['Team 1'] == 0)]
actions_1 = [old_data['Winning Team'], old_data['Losing Team']]
actions_w = ['W', 'L']

conds_2 = [(old_data['Team 1'] == 0),(old_data['Team 1'] == 1)]
actions_2 = [old_data['Winning Team'], old_data['Losing Team']]

old_data['Win?'] = np.select(conds_1,actions_w)
old_data['Team 2'] = np.select(conds_2,actions_2,default='Other')
old_data['Team 1'] = np.select(conds_1,actions_1,default='Other')
old_data = old_data.drop(columns = ['WTeamID',
                                    'LTeamID',
                                   'Winning Team',
                                   'Losing Team'])
#old_data = old_data[old_data['Season'] >= 2011]
old_data

,Season,WScore,LScore,Team 1,Team 2,Win?
0,2003,92,84,UNC Asheville,TX Southern,W
98,2003,95,84,Syracuse,Texas,W
342,2003,78,75,Arizona,Kansas,L
1018,2003,87,61,Wagner,Pittsburgh,L
358,2003,65,54,Oklahoma,Butler,W
...,...,...,...,...,...,...
687,2019,78,77,New Mexico St,Auburn,L
402,2019,61,51,Michigan St,Texas Tech,L
946,2019,62,58,Kentucky,Houston,W
942,2019,78,58,Texas Tech,Buffalo,W


In [40]:
pd.DataFrame(old_data['Team 2'].unique()).to_clipboard()
old_data

,Season,WScore,LScore,Team 1,Team 2,Win?
0,2003,92,84,UNC Asheville,TX Southern,W
98,2003,95,84,Syracuse,Texas,W
342,2003,78,75,Arizona,Kansas,L
1018,2003,87,61,Wagner,Pittsburgh,L
358,2003,65,54,Oklahoma,Butler,W
...,...,...,...,...,...,...
687,2019,78,77,New Mexico St,Auburn,L
402,2019,61,51,Michigan St,Texas Tech,L
946,2019,62,58,Kentucky,Houston,W
942,2019,78,58,Texas Tech,Buffalo,W


In [4]:
#scraping function to get SOS
def scrape(url,
           xpath = "//*[@id='meta']/div[2]/p[6]/text()[1]"):
    sleep(2)
    page = requests.get(url)

    tree = html.fromstring(page.content)  

    
    result = tree.xpath("//*[@id='meta']/div[2]/p[6]/text()[1]")
    return result[0][1:6]



In [6]:
links = pd.read_excel('page_links.xlsx')
links['SOS'] = [ scrape(x) for x in links['link-default'] ]
connect = pd.read_excel('name_connection.xlsx')
links = links.merge(connect, 
                    left_on = 'lower',
                    right_on = 'Sportsref')
pickle.dump(links,open(os.path.join(filepath, 'links'),'wb'))
links

,Year,lower,link-default,SOS,kenpom,Sportsref,Kaggle
0,2003,north-carolina-asheville,https://www.sports-reference.com/cbb/schools/n...,-5.60,UNC Asheville,north-carolina-asheville,UNC Asheville
1,2011,north-carolina-asheville,https://www.sports-reference.com/cbb/schools/n...,-2.89,UNC Asheville,north-carolina-asheville,UNC Asheville
2,2012,north-carolina-asheville,https://www.sports-reference.com/cbb/schools/n...,-2.71,UNC Asheville,north-carolina-asheville,UNC Asheville
3,2016,north-carolina-asheville,https://www.sports-reference.com/cbb/schools/n...,-3.25,UNC Asheville,north-carolina-asheville,UNC Asheville
4,2003,texas,https://www.sports-reference.com/cbb/schools/t...,18.91,Texas,texas,Texas
...,...,...,...,...,...,...,...
1127,2018,lipscomb,https://www.sports-reference.com/cbb/schools/l...,-3.80,Lipscomb,lipscomb,Lipscomb
1128,2019,colgate,https://www.sports-reference.com/cbb/schools/c...,-3.83,Colgate,colgate,Colgate
1129,2019,abilene-christian,https://www.sports-reference.com/cbb/schools/a...,-7.34,Abilene Christian,abilene-christian,Abilene Chr
1130,2019,gardner-webb,https://www.sports-reference.com/cbb/schools/g...,-4.43,Gardner Webb,gardner-webb,Gardner Webb


In [6]:
# I already wrote pickle, don't really want to write it again so I'll just scrape it 
links = pd.read_pickle(f'links')

In [33]:
def get_stats(team, year):
    #find the right row in the links df to get the link
    right_row = links[(links['Year'] == year) &
                      (links['Kaggle'] == team)]
    right_row = right_row.reset_index()
    right_link = right_row['link-default'][0]
    right_link

    #read the data
    sports_ref_data = pd.read_html(right_link)

    #two tables we need from the site
    basic_stats = sports_ref_data[1]


    #get the differences in basic statistics between team and Opp
    basic_stats = basic_stats.loc[:, basic_stats.columns!='Unnamed: 0']
    bad_df = basic_stats.index.isin([1,3])
    basic_stats = basic_stats[~bad_df]
    basic_stats = basic_stats.apply(pd.to_numeric, errors='coerce')
   

    #basic basketball stats
    data_export = pd.DataFrame(basic_stats.iloc[0][['2P','3P', '3P%','FT',
                               'ORB','TRB','AST','STL',
                              'BLK','TOV','PF','PTS']]).transpose()
    
    #include varper and ts%
    #data_export['stdvarPER'] = std_var_per
    #data_export['Weight TS%'] =  weighted_ts*100
    
    data_export['PA'] = basic_stats.iloc[1,-1]
    
    data_export['SOS'] = right_row['SOS']
    

    
    #pythagoras
    data_export['Pyth'] = (basic_stats.iloc[0,-1]**16.5)/(basic_stats.iloc[0,-1]**16.5 + basic_stats.iloc[1,-1]**16.5)
    
    #data_export['Guard Usage'] = guards
    return data_export

get_stats('UNC Asheville', 2003)

,2P,3P,3P%,FT,ORB,TRB,AST,STL,BLK,TOV,PF,PTS,PA,SOS,Pyth
0,18.4,6.6,0.362,15.8,12.3,35.7,13.4,7.2,3.1,16.1,19.3,72.5,78.6,-5.60,0.208671


In [41]:
final_df = pd.DataFrame()
for i in range(len(links)):
    year = links.iloc[i,:]['Year']
    team = links.iloc[i,:]['Kaggle']
    

    
    final_df = final_df.append(get_stats(team, year))
    sleep(3)
final_df.insert(0, "Year", list(links['Year']))
final_df.insert(1, "Team", list(links['Kaggle']))

final_df


,Year,Team,2P,3P,3P%,FT,ORB,TRB,AST,STL,BLK,TOV,PF,PTS,PA,SOS,Pyth
0,2003,UNC Asheville,18.4,6.6,0.362,15.8,12.3,35.7,13.4,7.2,3.1,16.1,19.3,72.5,78.6,-5.60,0.208671
0,2011,UNC Asheville,21.0,4.9,0.326,16.9,11.8,35.7,14.5,9.1,4.5,14.9,19.5,73.7,68.6,-2.89,0.765526
0,2012,UNC Asheville,20.5,6.6,0.355,20.1,11.1,32.1,16.6,8.3,2.6,13.4,18.0,80.7,71.4,-2.71,0.882909
0,2016,UNC Asheville,20.7,5.7,0.323,16.5,11.6,37.6,12.4,9.2,3.3,13.1,17.3,75.1,68.0,-3.25,0.837353
0,2003,Texas,21.8,5.9,0.356,17.9,16.1,42.0,14.5,6.2,3.7,13.1,20.5,79.3,69.7,18.91,0.893702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2018,Lipscomb,19.9,8.1,0.328,18.0,10.5,38.7,15.7,6.5,3.1,14.9,19.9,82.1,77.7,-3.80,0.712768
0,2019,Colgate,18.1,9.1,0.393,11.9,9.2,35.4,15.5,6.1,3.5,13.0,15.6,75.7,70.2,-3.83,0.776362
0,2019,Abilene Chr,19.0,7.4,0.380,13.4,9.6,32.6,15.4,8.7,2.7,12.0,18.7,73.6,63.6,-7.34,0.917550
0,2019,Gardner Webb,19.3,8.0,0.391,15.1,7.7,34.0,14.3,6.8,2.9,11.6,16.0,77.7,70.5,-4.43,0.832647


In [42]:
#deal with poll data


    
def poll_stats(team,year):
    #find the right row in the links df to get the link
    right_row = links[(links['Year'] == year) &
                      (links['Kaggle'] == team)]
    right_row = right_row.reset_index()
    right_link = right_row['link-default'][0][:-5]+str('-schedule.html')


    #read the data
    poll = pd.read_html(right_link)
    a = poll[0].transpose()
    a.columns = ['Brendan']
    return len(a) - pd.to_numeric(a['Brendan'], errors='coerce').isna().sum()


empty = []
for i in range(len(links)):
    sleep(3)
    year = links.iloc[i,:]['Year']
    team = links.iloc[i,:]['Kaggle']
    empty.append(poll_stats(team, year))

poll_data = pd.DataFrame()
poll_data['Year'] = list(links['Year'])
poll_data['Team'] = list(links['Kaggle'])
poll_data['Weeks Ranked'] = empty
poll_data



,Year,Team,Weeks Ranked
0,2003,UNC Asheville,0
1,2011,UNC Asheville,0
2,2012,UNC Asheville,0
3,2016,UNC Asheville,0
4,2003,Texas,19
...,...,...,...
1127,2018,Lipscomb,0
1128,2019,Colgate,0
1129,2019,Abilene Chr,0
1130,2019,Gardner Webb,0


In [43]:
#now merge the final dataframe with the poll data

final_df = final_df.merge(poll_data, left_on = ['Year','Team'], right_on = ['Year','Team'])
final_df

,Year,Team,2P,3P,3P%,FT,ORB,TRB,AST,STL,BLK,TOV,PF,PTS,PA,SOS,Pyth,Weeks Ranked
0,2003,UNC Asheville,18.4,6.6,0.362,15.8,12.3,35.7,13.4,7.2,3.1,16.1,19.3,72.5,78.6,-5.60,0.208671,0
1,2011,UNC Asheville,21.0,4.9,0.326,16.9,11.8,35.7,14.5,9.1,4.5,14.9,19.5,73.7,68.6,-2.89,0.765526,0
2,2012,UNC Asheville,20.5,6.6,0.355,20.1,11.1,32.1,16.6,8.3,2.6,13.4,18.0,80.7,71.4,-2.71,0.882909,0
3,2016,UNC Asheville,20.7,5.7,0.323,16.5,11.6,37.6,12.4,9.2,3.3,13.1,17.3,75.1,68.0,-3.25,0.837353,0
4,2003,Texas,21.8,5.9,0.356,17.9,16.1,42.0,14.5,6.2,3.7,13.1,20.5,79.3,69.7,18.91,0.893702,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1127,2018,Lipscomb,19.9,8.1,0.328,18.0,10.5,38.7,15.7,6.5,3.1,14.9,19.9,82.1,77.7,-3.80,0.712768,0
1128,2019,Colgate,18.1,9.1,0.393,11.9,9.2,35.4,15.5,6.1,3.5,13.0,15.6,75.7,70.2,-3.83,0.776362,0
1129,2019,Abilene Chr,19.0,7.4,0.380,13.4,9.6,32.6,15.4,8.7,2.7,12.0,18.7,73.6,63.6,-7.34,0.917550,0
1130,2019,Gardner Webb,19.3,8.0,0.391,15.1,7.7,34.0,14.3,6.8,2.9,11.6,16.0,77.7,70.5,-4.43,0.832647,0


In [30]:
#create kenpom df for AdjEm, O_rtg, D_rtg variables

kenpom_df = pd.DataFrame()
for i in range(2003,2020):
    
    url = str('https://kenpom.com/index.php?y='+str(i)+'&s=TeamName')

    header = {
      "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
      "X-Requested-With": "XMLHttpRequest"
    }

    r = requests.get(url, headers=header)

    dfs = pd.read_html(r.text)
    a = dfs[0][['Unnamed: 1_level_0','Unnamed: 4_level_0','Unnamed: 5_level_0', 'Unnamed: 7_level_0']]
    a.columns = ['Team','AdjEm', 'O_rtg','D_rtg']
    #a.iloc[:,1:] = a.iloc[:,1:].apply(pd.to_numeric, errors='coerce')
    a.insert(0,"Year",i)
    kenpom_df = kenpom_df.append(a)
kenpom_df['Team'] = kenpom_df['Team'].str.replace(' \d+', '')

kenpom_df

,Year,Team,AdjEm,O_rtg,D_rtg
0,2003,Air Force,+1.27,105.2,104.0
1,2003,Akron,-0.92,110.0,110.9
2,2003,Alabama,+14.60,110.3,95.7
3,2003,Alabama A&M,-18.25,90.0,108.2
4,2003,Alabama St.,-9.95,90.9,100.9
...,...,...,...,...,...
364,2019,Wright St.,+5.02,107.1,102.0
365,2019,Wyoming,-13.86,95.1,108.9
366,2019,Xavier,+10.60,111.1,100.5
367,2019,Yale,+8.79,111.6,102.8


In [56]:

'''
now, we have to make name associations. I have created 
kenpom_names.xlsx to show a connection between the names
that we have for the final_df, and the names we scraped.

I did something similar earlier to link the Kaggle data that was
scraped, and the names of the teams on sportsreference.com

Now, we merge kenpom_df with the name connector and merge that 
with the giant data scrape

'''
kenpom_df = pd.read_pickle('kenpom_df')
kp_names = pd.read_excel('kenpom_names.xlsx')
kp_names['Kenpom']
kenpom_df

x = kenpom_df.merge(kp_names,
                    left_on = ['Team'],
                    right_on = ['Kenpom'])
final_data = final_df.merge(x,
            left_on = ['Year', 'Team'],
              right_on = ['Year', 'Kaggle'])

# seeds = pd.read_excel('seeds_wiki.xlsx')
# final_data.merge(seeds,left_on = ['Year', 'Kaggle_x'],
#              right_on = ['Year', 'Kaggle']).drop_duplicates()
# final_data = final_data.drop(columns = ['Team_y','Kaggle_y','Wiki'])
# final_data
final_data

,Year,Team_x,2P,3P,3P%,FT,ORB,TRB,AST,STL,...,SOS,Pyth,Weeks Ranked,Team_y,AdjEm,O_rtg,D_rtg,Kenpom,Sportsref,Kaggle
0,2003,UNC Asheville,18.4,6.6,0.362,15.8,12.3,35.7,13.4,7.2,...,-5.60,0.208671,0,UNC Asheville,-12.35,101.2,113.5,UNC Asheville,north-carolina-asheville,UNC Asheville
1,2011,UNC Asheville,21.0,4.9,0.326,16.9,11.8,35.7,14.5,9.1,...,-2.89,0.765526,0,UNC Asheville,+1.91,101.9,100.0,UNC Asheville,north-carolina-asheville,UNC Asheville
2,2012,UNC Asheville,20.5,6.6,0.355,20.1,11.1,32.1,16.6,8.3,...,-2.71,0.882909,0,UNC Asheville,+5.50,110.4,104.9,UNC Asheville,north-carolina-asheville,UNC Asheville
3,2016,UNC Asheville,20.7,5.7,0.323,16.5,11.6,37.6,12.4,9.2,...,-3.25,0.837353,0,UNC Asheville,+3.44,103.6,100.1,UNC Asheville,north-carolina-asheville,UNC Asheville
4,2003,Texas,21.8,5.9,0.356,17.9,16.1,42.0,14.5,6.2,...,18.91,0.893702,19,Texas,+23.49,119.3,95.8,Texas,texas,Texas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,2018,Lipscomb,19.9,8.1,0.328,18.0,10.5,38.7,15.7,6.5,...,-3.80,0.712768,0,Lipscomb,+0.43,103.2,102.8,Lipscomb,lipscomb,Lipscomb
1123,2019,Colgate,18.1,9.1,0.393,11.9,9.2,35.4,15.5,6.1,...,-3.83,0.776362,0,Colgate,+4.60,110.5,105.9,Colgate,colgate,Colgate
1124,2019,Abilene Chr,19.0,7.4,0.380,13.4,9.6,32.6,15.4,8.7,...,-7.34,0.917550,0,Abilene Christian,+1.35,103.4,102.0,Abilene Christian,abilene-christian,Abilene Chr
1125,2019,Gardner Webb,19.3,8.0,0.391,15.1,7.7,34.0,14.3,6.8,...,-4.43,0.832647,0,Gardner Webb,-0.04,107.8,107.8,Gardner Webb,gardner-webb,Gardner Webb


In [54]:
'''
Now we connect final data and old data. 
Old data was downloaded from Kaggle, which is why we
kept that variable with us. If we can match Season/Year and 
Team1, Team2 with Kaggle, we are solid. 

# '''
# final_data
# # a lot of our metrics weren't available before 2011
# old_data = old_data[old_data['Season'] >= 2011]

# old_data
# missing_list = []
# for i in range(len(old_data)):
#     year = old_data.iloc[i,:]['Season']
#     team = old_data.iloc[i,:]['Team 1']
#     missing_list.append(len(final_data[(final_data['Year'] == year)&(final_data['Kaggle_y'] == team)]))
# #old_data.iloc[0,:]

# missing_list

# #len((final_data[(final_data['Year'] == 2011)&(final_data['Kaggle'] == 'Old Dominion')]))
# final_data.columns
final_df.merge(x, 
               left_on = ['Year', 'Team'],
              right_on = ['Year', 'Kaggle'])

,Year,Team_x,2P,3P,3P%,FT,ORB,TRB,AST,STL,...,SOS,Pyth,Weeks Ranked,Team_y,AdjEm,O_rtg,D_rtg,Kenpom,Sportsref,Kaggle
0,2003,UNC Asheville,18.4,6.6,0.362,15.8,12.3,35.7,13.4,7.2,...,-5.60,0.208671,0,UNC Asheville,-12.35,101.2,113.5,UNC Asheville,north-carolina-asheville,UNC Asheville
1,2011,UNC Asheville,21.0,4.9,0.326,16.9,11.8,35.7,14.5,9.1,...,-2.89,0.765526,0,UNC Asheville,+1.91,101.9,100.0,UNC Asheville,north-carolina-asheville,UNC Asheville
2,2012,UNC Asheville,20.5,6.6,0.355,20.1,11.1,32.1,16.6,8.3,...,-2.71,0.882909,0,UNC Asheville,+5.50,110.4,104.9,UNC Asheville,north-carolina-asheville,UNC Asheville
3,2016,UNC Asheville,20.7,5.7,0.323,16.5,11.6,37.6,12.4,9.2,...,-3.25,0.837353,0,UNC Asheville,+3.44,103.6,100.1,UNC Asheville,north-carolina-asheville,UNC Asheville
4,2003,Texas,21.8,5.9,0.356,17.9,16.1,42.0,14.5,6.2,...,18.91,0.893702,19,Texas,+23.49,119.3,95.8,Texas,texas,Texas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,2018,Lipscomb,19.9,8.1,0.328,18.0,10.5,38.7,15.7,6.5,...,-3.80,0.712768,0,Lipscomb,+0.43,103.2,102.8,Lipscomb,lipscomb,Lipscomb
1123,2019,Colgate,18.1,9.1,0.393,11.9,9.2,35.4,15.5,6.1,...,-3.83,0.776362,0,Colgate,+4.60,110.5,105.9,Colgate,colgate,Colgate
1124,2019,Abilene Chr,19.0,7.4,0.380,13.4,9.6,32.6,15.4,8.7,...,-7.34,0.917550,0,Abilene Christian,+1.35,103.4,102.0,Abilene Christian,abilene-christian,Abilene Chr
1125,2019,Gardner Webb,19.3,8.0,0.391,15.1,7.7,34.0,14.3,6.8,...,-4.43,0.832647,0,Gardner Webb,-0.04,107.8,107.8,Gardner Webb,gardner-webb,Gardner Webb


In [58]:
#for now, 5 rows didn't read right. This function should take care of that 
def viable_row(year, team1, team2):
    if len(final_data[(final_data['Year'] == year)&(final_data['Kaggle'] == team1)]) + len(final_data[(final_data['Year'] == year)&(final_data['Kaggle'] == team2)]) == 2:
        return True
    else:
        return False

old_data = old_data.reset_index(drop = True)
final_data

,Year,Team_x,2P,3P,3P%,FT,ORB,TRB,AST,STL,...,SOS,Pyth,Weeks Ranked,Team_y,AdjEm,O_rtg,D_rtg,Kenpom,Sportsref,Kaggle
0,2003,UNC Asheville,18.4,6.6,0.362,15.8,12.3,35.7,13.4,7.2,...,-5.60,0.208671,0,UNC Asheville,-12.35,101.2,113.5,UNC Asheville,north-carolina-asheville,UNC Asheville
1,2011,UNC Asheville,21.0,4.9,0.326,16.9,11.8,35.7,14.5,9.1,...,-2.89,0.765526,0,UNC Asheville,+1.91,101.9,100.0,UNC Asheville,north-carolina-asheville,UNC Asheville
2,2012,UNC Asheville,20.5,6.6,0.355,20.1,11.1,32.1,16.6,8.3,...,-2.71,0.882909,0,UNC Asheville,+5.50,110.4,104.9,UNC Asheville,north-carolina-asheville,UNC Asheville
3,2016,UNC Asheville,20.7,5.7,0.323,16.5,11.6,37.6,12.4,9.2,...,-3.25,0.837353,0,UNC Asheville,+3.44,103.6,100.1,UNC Asheville,north-carolina-asheville,UNC Asheville
4,2003,Texas,21.8,5.9,0.356,17.9,16.1,42.0,14.5,6.2,...,18.91,0.893702,19,Texas,+23.49,119.3,95.8,Texas,texas,Texas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,2018,Lipscomb,19.9,8.1,0.328,18.0,10.5,38.7,15.7,6.5,...,-3.80,0.712768,0,Lipscomb,+0.43,103.2,102.8,Lipscomb,lipscomb,Lipscomb
1123,2019,Colgate,18.1,9.1,0.393,11.9,9.2,35.4,15.5,6.1,...,-3.83,0.776362,0,Colgate,+4.60,110.5,105.9,Colgate,colgate,Colgate
1124,2019,Abilene Chr,19.0,7.4,0.380,13.4,9.6,32.6,15.4,8.7,...,-7.34,0.917550,0,Abilene Christian,+1.35,103.4,102.0,Abilene Christian,abilene-christian,Abilene Chr
1125,2019,Gardner Webb,19.3,8.0,0.391,15.1,7.7,34.0,14.3,6.8,...,-4.43,0.832647,0,Gardner Webb,-0.04,107.8,107.8,Gardner Webb,gardner-webb,Gardner Webb


In [62]:
collective_frame = pd.DataFrame()
for i in range(len(old_data)):
    game_row = old_data.iloc[i,:]

    year,team_1, team_2 = old_data.iloc[i,0],old_data.iloc[i,3],old_data.iloc[i,4]

    if viable_row(year, team_1, team_2) == True:
        team_1_df = final_data[(final_data['Year'] == year)&(final_data['Kaggle'] == team_1)].iloc[:,2:-3].drop(columns = ['Team_y'])
        team_2_df = final_data[(final_data['Year'] == year)&(final_data['Kaggle'] == team_2)].iloc[:,2:-3].drop(columns = ['Team_y'])

        num_t1_preds = len(list(team_1_df.columns))
        num_t2_preds = len(list(team_2_df.columns))
        datalist = [team_1_df.iloc[0,j] for j in range(num_t1_preds)] + [team_2_df.iloc[0,j] for j in range(num_t1_preds)]
        datalist.insert(0,year)
        datalist.insert(1,team_1)
        datalist.insert(2,team_2)
        datalist.insert(3+2*len(list(team_1_df.columns)),game_row['Win?'])
        x = pd.DataFrame(datalist).transpose()
        collective_frame = pd.concat([collective_frame,x])
        
namelist = ['year','team_1','team_2'] + [str(i)+' team 1' for i in (list(team_1_df.columns))] + [str(i)+' team 2' for i in (list(team_2_df.columns))] + ['W/L']
new_names_map = {x.columns[k]:namelist[k] for k in range(len(namelist))}
collective_frame.rename(new_names_map, axis=1, inplace=True)


conds = [(collective_frame['W/L'] == 'W'),(collective_frame['W/L'] == 'L')]
actions = [1, 0]
collective_frame['W/L'] = np.select(conds,actions)
collective_frame

collective_frame

,year,team_1,team_2,2P team 1,3P team 1,3P% team 1,FT team 1,ORB team 1,TRB team 1,AST team 1,...,PF team 2,PTS team 2,PA team 2,SOS team 2,Pyth team 2,Weeks Ranked team 2,AdjEm team 2,O_rtg team 2,D_rtg team 2,W/L
0,2003,UNC Asheville,TX Southern,18.4,6.6,0.362,15.8,12.3,35.7,13.4,...,18.4,73.2,71.5,-11.2,0.595733,0,-10.45,98.3,108.7,1
0,2003,Syracuse,Texas,23.8,5.3,0.344,16.0,13.7,40.7,14.9,...,20.5,79.3,69.7,18.91,0.893702,19,+23.49,119.3,95.8,1
0,2003,Arizona,Kansas,23.3,7.1,0.353,17.2,15.0,42.9,17.8,...,16.7,82.7,66.9,25.00,0.970641,19,+28.62,115.0,86.4,0
0,2003,Wagner,Pittsburgh,20.0,7.0,0.382,14.0,11.9,34.3,16.1,...,17.9,74.9,59.2,21.49,0.979793,19,+28.61,114.8,86.2,0
0,2003,Oklahoma,Butler,17.7,7.4,0.392,12.8,12.2,36.8,14.2,...,16.0,67.5,60.2,2.24,0.868586,0,+16.09,115.4,99.3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2019,New Mexico St,Auburn,18.2,9.3,0.334,13.8,12.7,38.4,14.7,...,18.3,79.7,68.8,20.84,0.918832,14,+25.00,120.9,95.9,0
0,2019,Michigan St,Texas Tech,19.3,8.2,0.378,14.5,10.6,40.5,18.3,...,17.4,72.8,59.5,22.79,0.965401,17,+30.03,114.1,84.1,0
0,2019,Kentucky,Houston,20.6,5.8,0.354,17.2,11.5,38.6,13.5,...,19.1,75.3,61.0,18.91,0.969968,15,+24.13,115.3,91.2,1
0,2019,Texas Tech,Buffalo,18.8,7.3,0.365,13.4,8.3,34.1,13.6,...,18.3,84.4,70.8,15.56,0.947806,19,+19.85,115.2,95.4,1


In [63]:
pickle.dump(collective_frame,open(os.path.join(filepath, '2003-2019_collective'),'wb'))

In [71]:
subtracting_frame = pd.DataFrame()
for i in range(len(old_data)):
    game_row = old_data.iloc[i,:]
    year,team_1, team_2 = old_data.iloc[i,0],old_data.iloc[i,3],old_data.iloc[i,4]
    if viable_row(year, team_1, team_2) == True:
        team_1_df = final_data[(final_data['Year'] == year)&(final_data['Kaggle'] == team_1)].iloc[:,2:-3].drop(columns = ['Team_y']).apply(pd.to_numeric, errors='coerce')
        team_2_df = final_data[(final_data['Year'] == year)&(final_data['Kaggle'] == team_2)].iloc[:,2:-3].drop(columns = ['Team_y']).apply(pd.to_numeric, errors='coerce')
        
        subtracted = team_1_df.reset_index(drop = True).subtract(team_2_df.reset_index(drop = True))
        
        subtracted.insert(0,'Year',year)
        subtracted.insert(1,'Team 1',team_1)
        subtracted.insert(2,'Team 2',team_2)
        subtracted.insert(len(subtracted.columns),'Win',old_data.iloc[i,-1])
        subtracting_frame = pd.concat([subtracting_frame,subtracted])
        
conds = [(subtracting_frame['Win'] == 'W'),(subtracting_frame['Win'] == 'L')]
actions = [1, 0]
subtracting_frame['Win'] = np.select(conds,actions)
subtracting_frame
        
subtracting_frame

,Year,Team 1,Team 2,2P,3P,3P%,FT,ORB,TRB,AST,...,PF,PTS,PA,SOS,Pyth,Weeks Ranked,AdjEm,O_rtg,D_rtg,Win
0,2003,UNC Asheville,TX Southern,-0.3,0.5,0.031,-1.5,-1.0,-2.5,-0.9,...,0.9,-0.7,7.1,5.60,-0.387062,0,-1.90,2.9,4.8,1
0,2003,Syracuse,Texas,2.0,-0.6,-0.012,-1.9,-2.4,-1.3,0.4,...,-3.6,0.3,-0.1,0.11,0.007897,-10,-0.21,-4.9,-4.7,1
0,2003,Arizona,Kansas,-3.0,2.3,0.018,1.6,0.5,1.1,0.6,...,0.8,2.5,3.8,-1.58,-0.014657,0,-1.82,0.6,2.4,0
0,2003,Wagner,Pittsburgh,-0.8,0.7,0.018,-0.5,0.7,-2.2,-1.7,...,-0.2,0.2,12.7,-27.97,-0.307521,-19,-29.77,-10.1,19.7,0
0,2003,Oklahoma,Butler,2.2,-0.9,0.000,1.2,4.3,8.8,1.5,...,2.2,2.9,-0.2,16.62,0.064651,19,6.92,-3.3,-10.2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2019,New Mexico St,Auburn,2.1,-2.1,-0.043,0.3,1.3,4.2,0.4,...,-0.2,-1.6,-4.3,-23.22,0.040346,-14,-11.38,-7.9,3.4,0
0,2019,Michigan St,Texas Tech,0.5,0.9,0.013,1.1,2.3,6.4,4.7,...,-0.8,4.8,5.5,2.14,-0.016406,3,0.78,6.9,6.1,0
0,2019,Kentucky,Houston,3.1,-3.2,-0.001,4.0,-0.6,-2.0,-1.3,...,-2.9,0.5,3.7,2.52,-0.038302,5,3.44,2.3,-1.1,1
0,2019,Texas Tech,Buffalo,-1.7,-2.3,0.028,-1.2,-4.3,-6.7,-3.0,...,-0.9,-11.6,-11.3,7.23,0.017594,-2,10.18,-1.1,-11.3,1


In [74]:
subtracting_frame

,Year,Team 1,Team 2,2P,3P,3P%,FT,ORB,TRB,AST,...,PF,PTS,PA,SOS,Pyth,Weeks Ranked,AdjEm,O_rtg,D_rtg,Win
0,2003,UNC Asheville,TX Southern,-0.3,0.5,0.031,-1.5,-1.0,-2.5,-0.9,...,0.9,-0.7,7.1,5.60,-0.387062,0,-1.90,2.9,4.8,1
0,2003,Syracuse,Texas,2.0,-0.6,-0.012,-1.9,-2.4,-1.3,0.4,...,-3.6,0.3,-0.1,0.11,0.007897,-10,-0.21,-4.9,-4.7,1
0,2003,Arizona,Kansas,-3.0,2.3,0.018,1.6,0.5,1.1,0.6,...,0.8,2.5,3.8,-1.58,-0.014657,0,-1.82,0.6,2.4,0
0,2003,Wagner,Pittsburgh,-0.8,0.7,0.018,-0.5,0.7,-2.2,-1.7,...,-0.2,0.2,12.7,-27.97,-0.307521,-19,-29.77,-10.1,19.7,0
0,2003,Oklahoma,Butler,2.2,-0.9,0.000,1.2,4.3,8.8,1.5,...,2.2,2.9,-0.2,16.62,0.064651,19,6.92,-3.3,-10.2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2019,New Mexico St,Auburn,2.1,-2.1,-0.043,0.3,1.3,4.2,0.4,...,-0.2,-1.6,-4.3,-23.22,0.040346,-14,-11.38,-7.9,3.4,0
0,2019,Michigan St,Texas Tech,0.5,0.9,0.013,1.1,2.3,6.4,4.7,...,-0.8,4.8,5.5,2.14,-0.016406,3,0.78,6.9,6.1,0
0,2019,Kentucky,Houston,3.1,-3.2,-0.001,4.0,-0.6,-2.0,-1.3,...,-2.9,0.5,3.7,2.52,-0.038302,5,3.44,2.3,-1.1,1
0,2019,Texas Tech,Buffalo,-1.7,-2.3,0.028,-1.2,-4.3,-6.7,-3.0,...,-0.9,-11.6,-11.3,7.23,0.017594,-2,10.18,-1.1,-11.3,1


In [70]:
pickle.dump(subtracting_frame,open(os.path.join(filepath, '2003-2019_subtracting_frame'),'wb'))

In [78]:
def binary(number):
    if number>=0:
        return 1
    else:
        return 0


binary_frame = subtracting_frame.copy()
data_cols = binary_frame.iloc[:,3:24].columns

binary_frame[data_cols]
for i in range(len(data_cols)):
    binary_frame[data_cols[i]] = binary_frame[data_cols[i]].apply(binary) 
    
conds = [(binary_frame['Win'] == 'W'),(binary_frame['Win'] == 'L')]
actions = [1, 0]
binary_frame['Win'] = np.select(conds,actions)
binary_frame

#pickle.dump(binary_frame,open(os.path.join(filepath, 'binary_frame'),'wb'))

binary_frame

,Year,Team 1,Team 2,2P,3P,3P%,FT,ORB,TRB,AST,...,PF,PTS,PA,SOS,Pyth,Weeks Ranked,AdjEm,O_rtg,D_rtg,Win
0,2003,UNC Asheville,TX Southern,0,1,1,0,0,0,0,...,1,0,1,1,0,1,0,1,1,0
0,2003,Syracuse,Texas,1,0,0,0,0,0,1,...,0,1,0,1,1,0,0,0,0,0
0,2003,Arizona,Kansas,0,1,1,1,1,1,1,...,1,1,1,0,0,1,0,1,1,0
0,2003,Wagner,Pittsburgh,0,1,1,0,1,0,0,...,0,1,1,0,0,0,0,0,1,0
0,2003,Oklahoma,Butler,1,0,1,1,1,1,1,...,1,1,0,1,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2019,New Mexico St,Auburn,1,0,0,1,1,1,1,...,0,0,0,0,1,0,0,0,1,0
0,2019,Michigan St,Texas Tech,1,1,1,1,1,1,1,...,0,1,1,1,0,1,1,1,1,0
0,2019,Kentucky,Houston,1,0,0,1,0,0,0,...,0,1,1,1,0,1,1,1,0,0
0,2019,Texas Tech,Buffalo,0,0,1,0,0,0,0,...,0,0,0,1,1,0,1,0,0,0


In [81]:
final_data.drop(columns = ['Kenpom','Sportsref','Kaggle','Team_y']).to_excel('each_team_data_2003-2019.xlsx')